In [ ]:
# cupy-cuda11xをインストールする
!pip install rich scipy numpy numba plotly kaleido cupy-cuda11x

In [ ]:
## download matrix
## 省略します

In [22]:
# 右辺ベクトル作成

import numpy as np
import scipy.io as sio

target="./s3dkt3m2.mtx"

def save_sparse_csr(filename, mat, b_vec):
    np.savez(filename,
             data=mat.data,
             indices=mat.indices,
             indptr=mat.indptr,
             b_vec=b_vec,
             shape=mat.shape)

(size_x, size_y, _, _, field, symmetry) = sio.mminfo(target)
mat = sio.mmread(target)

file_type = mat.getformat()
min_all = mat.min()
max_all = mat.max()
nnz = mat.getnnz()
nnz_row = mat.getnnz(1)
min_nnz = min(nnz_row)
max_nnz = max(nnz_row)
ave_nnz = np.average(nnz_row)
zero = float(size_x * size_x - nnz) / float(size_x * size_x) * 100.0

print("対称性 = %s" % symmetry)
print("格納形式 = %s" % file_type)
if symmetry == "symmetric":
    print("次元数 = %d" % (size_x * size_x))
else:
    print("X-サイズ = %d" % size_x)
    print("Y-サイズ = %d" % size_y)
print("非ゼロ要素数 = %d" % nnz)
print("ゼロ要素割合 = %.3f%%" % zero)
print("行最大非ゼロ要素数 = %d" % max_nnz)
print("行最小非ゼロ要素数 = %d" % min_nnz)
print("行平均非ゼロ要素数 = %d" % ave_nnz)
print("最大要素数 = %.16e" % max_all)
print("最小要素数 = %.16e" % min_all)

mat_csr = mat.tocsr()
x_vec = []
for _i in range(mat_csr.shape[0]):
    x_vec.append(1)
b_vec = mat_csr.dot(x_vec)
print("未知ベクトルを1になるように右辺ベクトルを生成　完了")

save_sparse_csr(target.replace(".mtx", ".npz"), mat_csr, b_vec)


対称性 = symmetric
格納形式 = coo
次元数 = 8181021601
非ゼロ要素数 = 3753461
ゼロ要素割合 = 99.954%
行最大非ゼロ要素数 = 42
行最小非ゼロ要素数 = 7
行平均非ゼロ要素数 = 41
最大要素数 = 3.2197282254561401e+03
最小要素数 = -1.1507613278490601e+03
未知ベクトルを1になるように右辺ベクトルを生成　完了


In [24]:
!python main.py -cg -t s3dkt3m2 -imax 400000 -e 1e-9 -np

[06/20/23 12:16:09] INFO     [2023-06-20 12:16:09,726][solver]: INFO  ]8;id=485915;file://main.py\main.py]8;;\:]8;id=205226;file://main.py#23\23]8;;\
                             - mat_file: ./matrix/s3dkt3m2.npz,                 
                             max_inter: 400000, epsilon:                        
                             1.000000e-09, numpy: True, numba: False,           
                             cupy: False                                        
                    INFO     [2023-06-20 12:16:09,976][solver]: INFO  ]8;id=68651;file://main.py\main.py]8;;\:]8;id=1240;file://main.py#31\31]8;;\
                             - CG method                                        
                    INFO     [2023-06-20 12:16:09,984][solver]: INFO  ]8;id=966383;file://main.py\main.py]8;;\:]8;id=469687;file://main.py#34\34]8;;\
                             - use numpy                                        
[06/20/23 13:31:41] INFO     [2023-06-20 13

In [25]:
!NUMBA_NUM_THREADS=2 python main.py -cg -t s3dkt3m2 -imax 400000 -e 1e-9 -nb

[06/20/23 13:31:45] INFO     [2023-06-20 13:31:45,894][solver]: INFO  ]8;id=611859;file://main.py\main.py]8;;\:]8;id=374655;file://main.py#23\23]8;;\
                             - mat_file: ./matrix/s3dkt3m2.npz,                 
                             max_inter: 400000, epsilon:                        
                             1.000000e-09, numpy: False, numba: True,           
                             cupy: False                                        
[06/20/23 13:31:46] INFO     [2023-06-20 13:31:46,096][solver]: INFO  ]8;id=540348;file://main.py\main.py]8;;\:]8;id=215589;file://main.py#31\31]8;;\
                             - CG method                                        
[06/20/23 14:19:28] INFO     [2023-06-20 14:19:28,286][solver]: INFO  ]8;id=467496;file://main.py\main.py]8;;\:]8;id=57584;file://main.py#48\48]8;;\
                             - is_converged = True, loop = 352312,              
                             time = 2862 

In [26]:
!NUMBA_NUM_THREADS=4 python main.py -cg -t s3dkt3m2 -imax 400000 -e 1e-9 -nb

[06/20/23 14:19:31] INFO     [2023-06-20 14:19:31,926][solver]: INFO  ]8;id=625599;file://main.py\main.py]8;;\:]8;id=85146;file://main.py#23\23]8;;\
                             - mat_file: ./matrix/s3dkt3m2.npz,                 
                             max_inter: 400000, epsilon:                        
                             1.000000e-09, numpy: False, numba: True,           
                             cupy: False                                        
[06/20/23 14:19:32] INFO     [2023-06-20 14:19:32,139][solver]: INFO  ]8;id=202499;file://main.py\main.py]8;;\:]8;id=412032;file://main.py#31\31]8;;\
                             - CG method                                        
[06/20/23 14:52:06] INFO     [2023-06-20 14:52:06,508][solver]: INFO  ]8;id=761977;file://main.py\main.py]8;;\:]8;id=631702;file://main.py#48\48]8;;\
                             - is_converged = True, loop = 344881,              
                             time = 1954 

In [27]:
!python main.py -cg -t s3dkt3m2 -imax 400000 -e 1e-9 -cp

[06/20/23 14:52:10] INFO     [2023-06-20 14:52:10,719][solver]: INFO  ]8;id=47623;file://main.py\main.py]8;;\:]8;id=518428;file://main.py#23\23]8;;\
                             - mat_file: ./matrix/s3dkt3m2.npz,                 
                             max_inter: 400000, epsilon:                        
                             1.000000e-09, numpy: False, numba:                 
                             False, cupy: True                                  
[06/20/23 14:52:11] INFO     [2023-06-20 14:52:11,425][solver]: INFO  ]8;id=510249;file://main.py\main.py]8;;\:]8;id=675858;file://main.py#31\31]8;;\
                             - CG method                                        
[06/20/23 14:57:18] INFO     [2023-06-20 14:57:18,462][solver]: INFO  ]8;id=27357;file://main.py\main.py]8;;\:]8;id=565192;file://main.py#48\48]8;;\
                             - is_converged = True, loop = 347218,              
                             time = 307 s 